importer fichier cvs

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

In [3]:
df = pd.read_csv('data.csv')

In [4]:
df['PromotionCompétitionSemaine'].fillna(0, inplace=True)

In [76]:
df.head()
df[df['PromotionCompétitionAnnée'] != df['Année']].head()

,Magasin,JourDeSemaine,Date,Ventes,Clients,Ouverture,PromotionsClassiques,JourFérié,VacancesScolaire,TypeDeMagasin,...,PromotionCompétitionAnnée,PromotionIntermédiaire,Mois,Trimestre,Année,Jour,Semaine,Saison,Mois-année,Ventes_quantile
942,2,5,2022-07-31,6064,625,1,1,0,1,0,...,2010.0,"Jan,Avr,Jul,Oct",7,3,2022,31,31,Ete,7-2022,NaN
943,2,4,2022-07-30,5567,601,1,1,0,1,0,...,2010.0,"Jan,Avr,Jul,Oct",7,3,2022,30,31,Ete,7-2022,NaN
944,2,3,2022-07-29,6402,727,1,1,0,1,0,...,2010.0,"Jan,Avr,Jul,Oct",7,3,2022,29,31,Ete,7-2022,NaN
945,2,2,2022-07-28,5671,646,1,1,0,1,0,...,2010.0,"Jan,Avr,Jul,Oct",7,3,2022,28,31,Ete,7-2022,NaN
946,2,1,2022-07-27,6627,638,1,1,0,1,0,...,2010.0,"Jan,Avr,Jul,Oct",7,3,2022,27,31,Ete,7-2022,NaN


In [6]:
df.dtypes

Magasin                          int64
JourDeSemaine                    int64
Date                            object
Ventes                           int64
Clients                          int64
Ouverture                        int64
PromotionsClassiques             int64
JourFérié                       object
VacancesScolaire                 int64
TypeDeMagasin                   object
ChoixArticles                   object
DistanceCompétition            float64
ConcoursOuvertMois             float64
ConcoursOuvertAnnée            float64
PromotionCompétition             int64
PromotionCompétitionSemaine    float64
PromotionCompétitionAnnée      float64
PromotionIntermédiaire          object
Mois                             int64
Trimestre                        int64
Année                            int64
Jour                             int64
Semaine                          int64
Saison                          object
dtype: object

In [ ]:
test = df['ConcoursOuvertAnnée'].apply(lambda x: True if x > 0 else False)

In [5]:
selected_columns = ['PromotionsClassiques','DistanceCompétition','VacancesScolaire']
df = df.dropna()
df['TypeDeMagasin'] = pd.Categorical(df['TypeDeMagasin']).codes


df[selected_columns].head(10)


,PromotionsClassiques,DistanceCompétition,VacancesScolaire
942,1,570.0,1
943,1,570.0,1
944,1,570.0,1
945,1,570.0,1
946,1,570.0,1
947,0,570.0,0
948,0,570.0,0
949,0,570.0,1
950,0,570.0,1
951,0,570.0,1


In [7]:
model_logistic = LogisticRegression(solver='lbfgs',random_state=0).fit(df[selected_columns], df['TypeDeMagasin'])
score = cross_val_score(model_logistic, df[selected_columns], df['TypeDeMagasin'], cv=5, scoring='accuracy')
print(f"score : {score}")
print(f"score moyen : {np.mean(score)}")
#print(f"score : {model_logistic.score(df[selected_columns], df['TypeDeMagasin'])}")
#print(f"prediction : { model_logistic.predict(df[selected_columns])}")
#print(f"predict_probat : {model_logistic.predict_proba(df[selected_columns])}")


score : 0.5422321984669746
prediction : [0 0 0 ... 0 0 0]
predict_probat : [[0.64743107 0.16745843 0.18511051]
 [0.64743107 0.16745843 0.18511051]
 [0.64743107 0.16745843 0.18511051]
 ...
 [0.57060031 0.178984   0.25041569]
 [0.57060031 0.178984   0.25041569]
 [0.57060031 0.178984   0.25041569]]


In [58]:
reg = LinearRegression()
#split data
x_train, x_test, y_train, y_test = train_test_split(df[selected_columns], df['Ventes'], test_size=0.2, random_state=0)
#train model`
reg.fit(x_train,y_train)
print(reg.score(x_test,y_test))
df_predict = pd.DataFrame(reg.predict(x_test), columns=['Predicted'])
df_predict['Actual'] = y_test.values
df_predict.head(10)


0.24588112114867644


,Predicted,Actual
0,4054.671138,5626
1,4490.204155,4210
2,7487.264893,6679
3,3951.676458,4803
4,3976.847668,3760
5,4413.908236,5687
6,3947.115952,0
7,3912.617125,3732
8,4415.288189,4870
9,7619.784685,12403


In [77]:
cross_val = cross_val_score(reg, df[selected_columns], df['Ventes'], cv=5)
print(cross_val)


[0.26671742 0.23120553 0.22138265 0.24246607 0.23410037]
